In [10]:
from psi_tools import label_gate, bin_to_qubits
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute

nx =3 
nl = nx 
na = nl + nx 

bin='011'

bounds = [0,2,4,6,6,7] # range(2**nx-1)

qx = QuantumRegister(nx, 'qx')
qanc=QuantumRegister(na, 'qanc') 
qlabel= QuantumRegister(nl, 'qlab')
#clab = ClassicalRegister(nl, 'clab')

qc = QuantumCircuit(qx, qanc, qlabel)
qc =qc.compose(bin_to_qubits(bin), qx)
lab_gate = label_gate(qc,qx, qlabel, qanc, bounds)
qc.append(lab_gate, [*qx, *qlabel, *qanc])

print(qc)

#qc.measure(qlabel, clab)
"""
backend = Aer.get_backend('aer_simulator')
job = execute(qc, backend, shots=10)

counts = job.result().get_counts()

print(counts)
"""

        ┌────────────────┐┌─────┐         ┌──────┐┌─────┐         ┌──────┐»
  qx_0: ┤0               ├┤0    ├─────────┤0     ├┤0    ├─────────┤0     ├»
        │                ││     │         │      ││     │         │      │»
  qx_1: ┤1 bin_to_qubits ├┤1    ├─────────┤1     ├┤1    ├─────────┤1     ├»
        │                ││     │         │      ││     │         │      │»
  qx_2: ┤2               ├┤2    ├─────────┤2     ├┤2    ├─────────┤2     ├»
        └────────────────┘│  P0 │         │  P0† ││  P1 │         │  P1† │»
qanc_0: ──────────────────┤3    ├─────────┤3     ├┤3    ├─────────┤3     ├»
                          │     │         │      ││     │         │      │»
qanc_1: ──────────────────┤4    ├─────────┤4     ├┤4    ├─────────┤4     ├»
                          │     │         │      ││     │         │      │»
qanc_2: ──────────────────┤5    ├────■────┤5     ├┤5    ├────■────┤5     ├»
              ┌───┐       └─────┘┌───┴───┐└──────┘└─────┘┌───┴───┐└──────┘»
qanc_3: ────

"\nbackend = Aer.get_backend('aer_simulator')\njob = execute(qc, backend, shots=10)\n\ncounts = job.result().get_counts()\n\nprint(counts)\n"

In [61]:
range(2**nx-1)

range(0, 7)